In [1]:
from mmengine import __version__
from mmcv import __version__ as mmcv_version
from mmdet import __version__ as mmdet_version
from torch import __version__ as torch_version
print(torch_version)
print(f"mmengine version: {__version__}")
print(f"mmcv version: {mmcv_version}")
print(f"mmdet version: {mmdet_version}")

2.0.0+cu118
mmengine version: 0.10.3
mmcv version: 2.0.1
mmdet version: 3.3.0


In [1]:
import os 
pwd= os.getcwd()
os.chdir('/home/jayant/Projects/mmdetection/latest/dataset')
from roboflow import Roboflow
rf = Roboflow(api_key="09fuHnf66MqJbmzHnAjP")
project = rf.workspace("dataset-t7hz7").project("cardboard-eupc8")
version = project.version(3)
dataset = version.download("coco-mmdetection")
os.chdir(pwd)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Cardboard-3 in coco-mmdetection:: 100%|██████████| 791/791 [00:00<00:00, 7999.48it/s]


In [28]:
from mmdet.utils import collect_env as collect_env_mmdet
collect_env_mmdet()


OrderedDict([('sys.platform', 'linux'),
             ('Python', '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'),
             ('CUDA available', True),
             ('MUSA available', False),
             ('numpy_random_seed', 2147483648),
             ('GPU 0', 'NVIDIA GeForce RTX 3060 Laptop GPU'),
             ('CUDA_HOME', '/usr/local/cuda'),
             ('NVCC', 'Cuda compilation tools, release 11.8, V11.8.89'),
             ('GCC',
              'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'),
             ('PyTorch', '2.0.0+cu118'),
             ('PyTorch compiling details',
              'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  -

In [27]:
from mmcv.utils import collect_env as collect_env_mmcv
collect_env_mmcv()

OrderedDict([('sys.platform', 'linux'),
             ('Python', '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'),
             ('CUDA available', True),
             ('MUSA available', False),
             ('numpy_random_seed', 2147483648),
             ('GPU 0', 'NVIDIA GeForce RTX 3060 Laptop GPU'),
             ('CUDA_HOME', '/usr/local/cuda'),
             ('NVCC', 'Cuda compilation tools, release 11.8, V11.8.89'),
             ('GCC',
              'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'),
             ('PyTorch', '2.0.0+cu118'),
             ('PyTorch compiling details',
              'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  -

In [4]:
from mmengine.config import Config

cfg = Config.fromfile('../mmdetection/configs/faster_rcnn/faster-rcnn_x101-64x4d_fpn_1x_coco.py')

In [22]:
cfg.model.roi_head.bbox_head.num_classes = 20
dataroot = "/workspace/objectdet/datasets/Logistics-2/"
cfg.data_root = dataroot

metainfo = {
    'classes': ('barcode', 'car', 'cardboard box', 'fire', 'forklift', 'freight container', 'gloves', 'helmet', 'ladder', 'license plate', 'person', 'qr code', 'road sign', 'safety vest', 'smoke', 'traffic cone', 'traffic light', 'truck', 'van', 'wood pallet'),
    'palette': [
        (220, 20, 60),
    ]
}

cfg.train_dataloader = dict(
    batch_size=2,
    num_workers=4,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=dataroot,
        metainfo=metainfo,
        ann_file='train/_annotations.coco.json',
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=cfg.train_pipeline,
        backend_args=cfg.backend_args))
cfg.val_dataloader = dict(
    batch_size=2,
    num_workers=4,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=dataroot,
        metainfo=metainfo,
        ann_file='valid/_annotations.coco.json',
        data_prefix=dict(img='valid/'),
        test_mode=True,
        pipeline=cfg.test_pipeline,
        backend_args=cfg.backend_args))

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=dataroot + 'valid/_annotations.coco.json',
    metric='bbox',
    format_only=False,
    backend_args=cfg.backend_args)


# inference on test dataset and
# format the output results for submission.
cfg.test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=dataroot,
        metainfo=metainfo,
        ann_file=dataroot + 'test/_annotations.coco.json',
        data_prefix=dict(img='test/'),
        test_mode=True,
        pipeline=cfg.test_pipeline))
cfg.test_evaluator = dict(
    type='CocoMetric',
    metric='bbox',
    format_only=True,
    ann_file=dataroot + 'test/_annotations.coco.json',
    outfile_prefix='/workspace/objectdet/work_dirs/faster_rcnn_640x640_1x1b_30e_r101_fpn_1x_logistics/test')



# cfg.param_scheduler = list(
# cfg.param_scheduler =  dict(type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=1)
#     dict(
#         type='MultiStepLR',
#         begin=0,
#         end=1,
#         by_epoch=True,
#         milestones=[8, 11],
#         gamma=0.1)
# )

cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=12, val_interval=3)
# cfg.param_scheduler[1].end = 2
# cfg.param_scheduler[0].end = 2
# cfg.param_scheduler[0].by_epoch = True
cfg.optim_wrapper = dict(
    type='OptimWrapper',
    # optimizer=dict(type='SGD', lr=0.00001, momentum=0.9, weight_decay=0)
    optimizer=dict(type='AdamW', lr=0.0002)
    ,)
# cfg.auto_scale_lr = dict(enable=True, base_batch_size=1)
cfg.work_dir = '/workspace/objectdet/work_dirs/faster_rcnn_640x640_1x1b_30e_r101_fpn_1x_logistics/1'

cfg.default_hooks = dict(
                checkpoint=dict(
                    type='CheckpointHook', 
                    interval=3,
                    max_keep_ckpts=2,  # only keep latest 2 checkpoints
                    save_best='auto'
                    ),
                logger=dict(type='LoggerHook', interval=100),
                
                visualization=dict(type='DetVisualizationHook'),
                )
# cfg.vis_backends = dict(type='LocalVisBackend')
# size of the images = 1333x800

In [23]:
cfg.dump('/workspace/objectdet/configs/faster_rcnn_640x640_1x1b_30e_r101_fpn_1x_logistics.py')

In [3]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

runner.train()

03/14 01:36:59 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 370235367
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode

FasterRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNeXt(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu)

In [23]:
from mmdet.apis import DetInferencer

model = DetInferencer(cfg,"/home/jayant/Projects/mmdetection/latest/work_dirs/faster_rcnn_r101_fpn_1x_boxes/2/best_coco_bbox_mAP_epoch_12.pth")

Loads checkpoint by local backend from path: /home/jayant/Projects/mmdetection/latest/work_dirs/faster_rcnn_r101_fpn_1x_boxes/2/best_coco_bbox_mAP_epoch_12.pth
03/08 12:52:58 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/jayant/Projects/mmdetection/latest/.venv/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [27]:
pred = model("/home/jayant/Projects/mmdetection/latest/images/brown-parcel-boxes.jpg",out_dir="/home/jayant/Projects/mmdetection/latest/outputs/1", )

Output()

In [11]:
!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve work_dirs/faster_rcnn_r101_fpn_1x_boxes/1/20240308_104650/vis_data/20240308_104650.json --keys loss_cls --legend loss_cls --out records/faster_rcnn_r101_fpn_1x_boxes/1/loss_cls.jpg --title "faster-rcnn/loss-cls" 

plot curve of work_dirs/faster_rcnn_r101_fpn_1x_boxes/1/20240308_104650/vis_data/20240308_104650.json, metric is loss_cls
save curve to: records/faster_rcnn_r101_fpn_1x_boxes/1/loss_cls.jpg


In [17]:
cfg.dump('/home/jayant/Projects/mmdetection/latest/mmdetection/configs/faster_rcnn/faster-rcnn_x101-64x4d_fpn_1x_boxes.py')


In [20]:
!python mmdetection/tools/test.py /home/jayant/Projects/mmdetection/latest/mmdetection/configs/faster_rcnn/faster-rcnn_x101-64x4d_fpn_1x_boxes.py work_dirs/faster_rcnn_r101_fpn_1x_boxes/2/best_coco_bbox_mAP_epoch_12.pth --out result_Potato_new1/result_1.pkl --cfg-options "jsonfile_prefix=./result_box_frrcnn" --work-dir /home/jayant/Projects/mmdetection/latest/work_dirs/faster_rcnn_r101_fpn_1x_boxes/2/tests

03/08 12:32:54 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 509035958
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode

In [21]:
!python mmdetection/tools/analysis_tools/analyze_results.py /home/jayant/Projects/mmdetection/latest/mmdetection/configs/faster_rcnn/faster-rcnn_x101-64x4d_fpn_1x_boxes.py /home/jayant/Projects/mmdetection/latest/result_boxes/result_1.pkl result_boxes --show 

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
03/08 12:37:51 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 547688622
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU

In [22]:
!python mmdetection/tools/analysis_tools/confusion_matrix.py /home/jayant/Projects/mmdetection/latest/mmdetection/configs/faster_rcnn/faster-rcnn_x101-64x4d_fpn_1x_boxes.py  /home/jayant/Projects/mmdetection/latest/result_boxes/result_1.pkl  result_boxes_1 --show

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 3349.1 task/s, elapsed: 0s, ETA:     0s/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/confusion_matrix.py:229: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
Figure(180x144)


In [ ]:
!python mmdetection//tools/analysis_tools/coco_error_analysis.py  --out abc_r.pdf